In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
subm = pd.read_csv('../input/sample_submission.csv')

In [3]:
train.shape

In [4]:
train.head()

In [7]:
test.shape

In [8]:
test.head()

In [12]:
subm.head()

In [13]:
subm.shape

In [15]:
train['comment_text'][0]

In [14]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [16]:
lens

In [17]:
lens.hist();

In [18]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train[label_cols].max(axis=1)



In [19]:
1-train[label_cols].max(axis=1)

In [21]:
train['none'] = 1-train[label_cols].max(axis=1)

In [22]:
train.shape

In [23]:
train.head()

In [24]:
train.describe()

In [28]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)


In [32]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [33]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [37]:
from sklearn.naive_bayes import MultinomialNB

In [47]:
nb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
preds = np.zeros((len(test), len(label_cols)))

In [48]:
preds.shape

In [52]:
for i, j in enumerate(label_cols):
    print('fit', j)
    #m,r = get_mdl(train[j])
    m= nb.fit(trn_term_doc, train[j])
    #preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    preds[:,i] = m.predict_proba(test_term_doc)[:,1]

In [54]:
preds

In [55]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)